In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

from google.colab import drive
drive.mount('drive')

# Read sentiment data from CSV file
data = pd.read_csv('/content/drive/My Drive/recommendation_data.csv')

# Read the recommendation data from CSV file
#data = pd.read_csv('recommendation_data.csv')


# Drop duplicate entries
data = data.drop_duplicates(subset=['User ID', 'Item ID'])

# Create the user-item matrix
user_item_matrix = data.pivot(index='User ID', columns='Item ID', values='Score').fillna(0)

# Compute item-item similarity matrix using cosine similarity
item_sim_matrix = cosine_similarity(user_item_matrix.T)

# Fit a k-Nearest Neighbors model on the item similarity matrix
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(item_sim_matrix)

# Function to recommend similar items based on collaborative filtering
def collaborative_filtering_recommendation(item_id, num_recommendations):
    item_index = data[data['Item ID'] == item_id].index[0]
    _, indices = knn_model.kneighbors(item_sim_matrix[item_index].reshape(1, -1), n_neighbors=num_recommendations+1)
    similar_items = data.iloc[indices[0][1:]]['Item ID'].values
    return similar_items

# Function to recommend items using hybrid recommendation
def hybrid_recommendation(user_id, num_recommendations):
    user_items = data[data['User ID'] == user_id]['Item ID']
    recommended_items = []

    for item in user_items:
        similar_items = collaborative_filtering_recommendation(item, num_recommendations)
        recommended_items.extend(similar_items)

    return recommended_items[:num_recommendations]

# Test the recommendation system
user_id = 20
num_recommendations = 5

recommended_items = hybrid_recommendation(user_id, num_recommendations)

# Print the recommended items
print(f"Recommended items for User ID {user_id}:")
for i, item in enumerate(recommended_items):
    print(f"{i+1}. Item ID: {item}")


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Recommended items for User ID 20:
1. Item ID: 2814
2. Item ID: 2816
3. Item ID: 3634
4. Item ID: 958
5. Item ID: 1623
